<a href="https://colab.research.google.com/github/surya-vikram/pytorch-lab/blob/main/fashionmnist_ann.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [74]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision.datasets import FashionMNIST
from torchvision.transforms import ToTensor

In [75]:
train_data = FashionMNIST(root='data',
                          train=True,
                          download=True,
                          transform=ToTensor())

test_data = FashionMNIST(root='data',
                         train=False,
                         download=True,
                         transform=ToTensor())

In [76]:
train_dataloader = DataLoader(dataset=train_data, batch_size=64)
test_dataloader = DataLoader(dataset=test_data, batch_size=64)

In [77]:
device = 'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'
device

'cuda'

In [78]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.input = nn.Flatten()
        self.network = nn.Sequential(
            nn.Linear(784, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 10)
        )

    def forward(self, x):
        x = self.input(x)
        logits = self.network(x)
        return logits

model = NeuralNetwork().to(device)
model

NeuralNetwork(
  (input): Flatten(start_dim=1, end_dim=-1)
  (network): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=256, bias=True)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=128, bias=True)
    (5): ReLU()
    (6): Linear(in_features=128, out_features=10, bias=True)
  )
)

In [79]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-2)

In [80]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [81]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [82]:
epochs = 8
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.308545  [   64/60000]
loss: 2.279143  [ 6464/60000]
loss: 2.224056  [12864/60000]
loss: 2.163687  [19264/60000]
loss: 1.929593  [25664/60000]
loss: 1.590305  [32064/60000]
loss: 1.392110  [38464/60000]
loss: 1.158264  [44864/60000]
loss: 1.103441  [51264/60000]
loss: 1.010524  [57664/60000]
Test Error: 
 Accuracy: 62.2%, Avg loss: 0.973032 

Epoch 2
-------------------------------
loss: 1.051050  [   64/60000]
loss: 1.022230  [ 6464/60000]
loss: 0.773208  [12864/60000]
loss: 0.892580  [19264/60000]
loss: 0.750218  [25664/60000]
loss: 0.726471  [32064/60000]
loss: 0.844232  [38464/60000]
loss: 0.758342  [44864/60000]
loss: 0.750991  [51264/60000]
loss: 0.743359  [57664/60000]
Test Error: 
 Accuracy: 71.7%, Avg loss: 0.767917 

Epoch 3
-------------------------------
loss: 0.717725  [   64/60000]
loss: 0.780496  [ 6464/60000]
loss: 0.534270  [12864/60000]
loss: 0.724184  [19264/60000]
loss: 0.641743  [25664/60000]
loss: 0.579173  [32064/600

In [83]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [84]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth", weights_only=True))

<All keys matched successfully>

In [86]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[-1][0], test_data[-1][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Sandal", Actual: "Sandal"
